# What Algorithm to choose?


![alt text](./img/cheat_sheet.png)

![alt text](./img/nuclear.jpg)

# XGBoost. The very best there is. When you absolutely, positively got to forecast everything in the room, accept no substitutes.

# The XGBoost Advantage

### Regularization:
Standard GBM implementation has no regularization like XGBoost, therefore it also helps to reduce overfitting.
### Parallel Processing:
XGBoost implements parallel processing and is blazingly faster as compared to GBM.
XGBoost also supports implementation on Hadoop.
### High Flexibility
XGBoost allow users to define custom optimization objectives and evaluation criteria.
### Handling Missing Values
XGBoost has an in-built routine to handle missing values.
### Tree Pruning:
A GBM would stop splitting a node when it encounters a negative loss in the split. XGBoost on the other hand make splits upto the max_depth specified and then start pruning the tree backwards and remove splits beyond which there is no positive gain.
### Built-in Cross-Validation
XGBoost allows user to run a cross-validation at each iteration of the boosting process and thus it is easy to get the exact optimum number of boosting iterations in a single run.

###### Courtesy of AARSHAY JAIN 
###### https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

In [ ]:
from traitlets.config.manager import BaseJSONConfigManager 
path = "/Users/dreyco676/.jupyter/nbconfig" 
cm = BaseJSONConfigManager(config_dir=path)
cm.update('livereveal', { 'scroll': True, 'width': 1280, 'height': 960,})

# Let's get started!

In [ ]:
from numpy import loadtxt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd
import numpy as np

# Import Feature Engineered Data

In [ ]:
# read data in
df = pd.read_csv('ml_ready_data.csv', index_col='Unnamed: 0', low_memory=False)
# CSVs are dumb, sometimes need to force things to numeric
cols = df.columns
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
df.shape

# What do we have for variables?

In [ ]:
list(df.columns.values)

# Building your first Model

![alt text](./img/dependent-vs-independent-variable1.jpg)

In [ ]:
# select X & Y
y = 'SalesClosePrice'
feature_names = list(df.columns.values)
feature_names.remove(y)

## Split Data into Test & Train

In [ ]:
# split data into train and test sets
Xtrain, Xtest, ytrain, ytest = train_test_split(df[feature_names], df[y], test_size=0.5, random_state=42)

![alt text](./img/train-test.png)

In [ ]:
# keep labels on test/train
X_train = pd.DataFrame(data=Xtrain, columns=feature_names)
Xtest = pd.DataFrame(data=Xtest, columns=feature_names)

dtrain = xgb.DMatrix(Xtrain, label=ytrain)

# Fit the Model to the training set

In [ ]:
model = xgb.XGBRegressor(n_jobs=4)
model.fit(Xtrain, ytrain)

# Evaluating your model, what Metrics to use?

* metrics.explained_variance_score(y_true, y_pred)	
    * Explained variance regression score function


* metrics.mean_absolute_error(y_true, y_pred)	       
    * Mean absolute error regression loss

* metrics.mean_squared_error(y_true, y_pred[, …])	    
    * Mean squared error regression loss


* metrics.mean_squared_log_error(y_true, y_pred)	    
    * Mean squared logarithmic error regression loss


* metrics.median_absolute_error(y_true, y_pred)	    
    * Median absolute error regression loss


* metrics.r2_score(y_true, y_pred[, …])	            
    * R^2 (coefficient of determination) regression score function.

# First evaluate your data

In [ ]:
# make predictions for test data
ypred = model.predict(Xtest)
predictions = [round(value) for value in ypred]

# R^2 Metric

In [ ]:
# evaluate predictions
r2_score(ytest, ypred)

# RMSE Metric

In [ ]:
from math import sqrt
rmse = sqrt(mean_squared_error(ytest, ypred))
rmse

# Difference between R^2 and RMSE?

Both indicate the goodness of the fit.

R-squared is conveniently scaled between 0 and 1, whereas RMSE is not scaled to any particular values. This can be good or bad; obviously R-squared can be more easily interpreted, but with RMSE we explicitly know how much our predictions deviate, on average, from the actual values in the dataset. So in a way, RMSE tells you more.

###### user3796494
###### https://stats.stackexchange.com/questions/142248/difference-between-r-square-and-rmse-in-linear-regression

# Tuning Your Model

In [ ]:
# Set the parameters by cross-validation
tuned_parameters = [{'subsample': [1], 'colsample_bytree': [1], 'gamma':[0], 
                     'learning_rate':[0.1,], 'max_depth': [4, 5, 6], 
                     'min_child_weight': [0.1, 0.5, 0.75]}]


print("# Tuning hyper-parameters for r2")
print()

model = GridSearchCV(xgb.XGBRegressor(n_jobs=4), tuned_parameters, cv=5, scoring='r2')
model.fit(Xtrain, ytrain)

print("Best parameters set found on development set:")
print()
print(model.best_params_)
print()
print("Grid scores on development set:")
print()
means = model.cv_results_['mean_test_score']
stds = model.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, model.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))

# Now we can explore the 'best' model we have

In [ ]:
# fit model no training data
model = xgb.XGBRegressor(max_depth=5, min_child_weight=0.1, n_jobs=4)
model.fit(Xtrain, ytrain)

# Make predictions for test data

In [ ]:
# make predictions for test data
ypred = model.predict(Xtest)
predictions = [round(value) for value in ypred]

In [ ]:
rmse = sqrt(mean_squared_error(ytest, ypred))
rmse

# What is important?

In [ ]:
from xgboost import plot_importance
%matplotlib inline
from matplotlib import pyplot as plt

# feature importance
fig, ax = plt.subplots(figsize=(40, 40))
plot_importance(model, color='red', grid=False, show_values=False, ax=ax)
plt.title('importance', fontsize = 36)
plt.yticks(fontsize = 18)
plt.ylabel('features', fontsize = 36)

# What can we afford to lose?

* Too many variables can slow run time of model
* Can hamper explainability
* May be hard to curate
* May not be reported in a timely manner

In [ ]:
# use feature importance for feature selection
from numpy import loadtxt
from numpy import sort
from sklearn.feature_selection import SelectFromModel

# fit model on all training data
model = xgb.XGBClassifier()
model.fit(Xtrain, ytrain)
# make predictions for test data and evaluate
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
r2 = r2_score(ytest, predictions)
print("r2: %.2f%%" % (r2))
# Fit model using each importance as a threshold
thresholds = sort(model.feature_importances_)
for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(Xtrain)
    # train model
    selection_model = xgb.XGBClassifier()
    selection_model.fit(select_X_train, ytrain)
    # eval model
    select_X_test = selection.transform(Xtest)
    y_pred = selection_model.predict(select_X_test)
    predictions = [round(value) for value in y_pred]
    r2 = r2_score(y_test, predictions)
    print("Thresh=%.3f, n=%d, r2: %.2f%%" % (thresh, select_X_train.shape[1], r2))

# What if we want to answer a different question?

In [ ]:
# del df['listing_to_median_ratio']
# del df['listing_price_per_SQFT']
# del df['OriginalListPrice']
# prep X & Y
y = 'SalesClosePrice'
feature_names = list(df.columns.values)
feature_names.remove(y)
# split data into train and test sets
Xtrain, Xtest, ytrain, ytest = train_test_split(df[feature_names], df[y], test_size=0.5)
# fit model no training data
model = xgb.XGBRegressor(max_depth=5, min_child_weight=0.1, n_jobs=4)
model.fit(Xtrain, ytrain)
# make predictions for test data
ypred = model.predict(Xtest)
predictions = [round(value) for value in ypred]
mse = mean_squared_error(ytest, ypred)
rmse = sqrt(mse)
rmse